In [1]:
# 1. Install libraries
!pip install numpy pandas tensorflow==2.0.0 scikit-learn==0.21.3

You should consider upgrading via the '/Users/zhang_family_mac/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
# 2. Import libaries
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
import os
import sys
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
from keras.models import *
from keras.layers import *
import pickle


if not sys.warnoptions:
    import warnings

    warnings.simplefilter("ignore")

Using TensorFlow backend.


In [4]:
# 3. Read data
def read_data():
    # SP500 + DOW30 + Nasdaq
    df_spy = pd.read_csv('/Users/zhang_family_mac/Yongqiang/stock_prediction/data/sp500_max.csv')
    df_dowjones = pd.read_csv('/Users/zhang_family_mac/Yongqiang/stock_prediction/data/dowjones_max.csv')
    df_nasdaq = pd.read_csv('/Users/zhang_family_mac/Yongqiang/stock_prediction/data/nasdaq_max.csv')
    df_spy = df_spy.set_index('Date').add_suffix('_spy')
    df_dowjones = df_dowjones.set_index('Date').add_suffix('_dowjones')
    df_nasdaq = df_nasdaq.set_index('Date').add_suffix('_nasdaq')
    df = df_spy.merge(df_dowjones, left_index=True, right_index=True, how='inner')
    df = df.merge(df_nasdaq, left_index=True, right_index=True, how='inner')
    df = df.reset_index()
    df = df[['Open_spy', 'Close_spy', 'Volume_spy', 'Low_spy', 'High_spy', \
             'Open_dowjones', 'Close_dowjones', 'Volume_dowjones', 'Low_dowjones', 'High_dowjones', \
             'Open_nasdaq', 'Close_nasdaq', 'Volume_nasdaq', 'Low_nasdaq', 'High_nasdaq']]
    return df

def TA_analysis(df):
    # TA Analysis #1: finta
    from finta import TA
    for method in TA.__dict__.keys():
        if method.startswith('__'):
            continue
        else:
            try:
                df[method] = getattr(TA, method)(df)
            except:
                print(method, 'cannot be called')
    import pandas_ta as ta
    df.ta.indicators()
    # df.ta.log_return(cumulative=True, append=True)
    # df.ta.percent_return(cumulative=True, append=True)
    
    return df

In [5]:
# 4. Feature Engineering
# Get data in the Kera's format
def series_to_supervised(data, n_in=1, n_out=1, lead_time=0, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(lead_time, lead_time + n_out):
        cols.append(df.iloc[:, 1].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (n_vars))]
        else:
            names += [('var%d(t+%d)' % (n_vars, i))]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

def feature_engineering(df, stage='TRAIN', model=None, feature_window=60, target_window=1, lead_time_window=0):
    # get time series data
    values = df.values
    # ensure all data is float
    values = values.astype('float32')
    # normalize features
    if stage == 'TRAIN':
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(values)
    else:
        scaler = model['scaler']
    scaled = scaler.transform(values)
    print('scaled: ', scaled.shape)

    # frame as supervised learning
    if stage == 'TRAIN':
        df_reframed = series_to_supervised(scaled, feature_window, target_window, lead_time_window, True)
    else:
        df_reframed = series_to_supervised(scaled, feature_window, 0, 0, True)

    print(df_reframed.head(10))
    print(len(df_reframed))
    return scaler, df_reframed

def split_data(df_reframed, train_ratio=1.0, target_window=1):
    # split into train and test sets
    dataset = df_reframed.values
    train_size = int(len(dataset) * train_ratio)
    train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

    # split into input and outputs
    train_X, train_y = train[:, :-target_window], train[:, -target_window:]
    test_X, test_y = test[:, :-target_window], test[:, -target_window:]
    return train_X, train_y, test_X, test_y


def reshape(X, feature_window=60):
    # reshape input to be 3D [samples, timesteps, features]
    num_features = int(X.shape[1] / feature_window)
    X = X.reshape((X.shape[0], feature_window, num_features))
    return X

In [6]:
# 5. Modeling
def LSTM_auto_encoder(train_X):
    inputs_ae = Input(shape=(train_X.shape[1], train_X.shape[2]))
    encoded_ae = LSTM(128, return_sequences=True, dropout=0.3)(inputs_ae, training=True)
    decoded_ae = LSTM(32, return_sequences=True, dropout=0.3)(encoded_ae, training=True)
    out_ae = TimeDistributed(Dense(train_X.shape[2]))(decoded_ae)

    sequence_autoencoder = Model(inputs_ae, out_ae)
    sequence_autoencoder.compile(optimizer='adam', loss='mse', metrics=['mse'])
    sequence_autoencoder.summary()

    sequence_autoencoder.fit(train_X, train_X, batch_size=1024, epochs=50, verbose=2, shuffle=True)

    encoder = Model(inputs_ae, encoded_ae)
    return encoder


def LSTM_forecaster(train_X, train_y, target_window=15):
    input_fc = Input(shape=(train_X.shape[1], train_X.shape[2]))
    lstm_fc = LSTM(128, return_sequences=True, dropout=0.3)(input_fc, training=True)
    lstm_fc = LSTM(32, return_sequences=False, dropout=0.3)(lstm_fc, training=True)
    dense_fc = Dense(50)(lstm_fc)
    out_fc = Dense(target_window)(dense_fc)

    model_fc = Model(input_fc, out_fc)

    model_fc.compile(loss='mse', optimizer='adam', metrics=['mse'])
    model_fc.fit(train_X, train_y, epochs=50, batch_size=1024, verbose=2, shuffle=True)
    return model_fc

def train(train_X, train_y, target_window=15):
    encoder = LSTM_auto_encoder(train_X)
    train_X = encoder.predict(train_X)
    forecaster = LSTM_forecaster(train_X, train_y, target_window)
    return encoder, forecaster


def predict(model, df):
    y_pred = model.predict(df)
    return y_pred


def evaluate(y_pred, y_true):
    y_diff = y_pred - y_true
    y_diff_square = y_diff * y_diff
    y_rmse = [np.sqrt(np.mean(y_diff[:, i])) for i in range(y_true.shape[1])]
    y_mape = np.mean(abs(y_diff))
    print('Evaluation result - rmse: ', y_rmse, ' mape: ', y_mape)

In [7]:
# 6. Workflow
def workflow_train(data_freq='60s', train_ratio=1.0, feature_window=60, target_window=1, lead_time_window=0, result_path=None):
    from numpy.random import seed
    seed(1)
    import tensorflow
    tensorflow.random.set_seed(2)
    
    df = read_data()
    scaler, df_reframed = feature_engineering(df, stage='TRAIN', model=None, feature_window=feature_window, target_window=target_window, lead_time_window=lead_time_window)
    train_X, train_y, test_X, test_y = split_data(df_reframed, train_ratio=train_ratio, target_window=target_window)
    train_X = reshape(train_X, feature_window)
    encoder, forecaster = train(train_X, train_y, target_window=target_window)
    if train_ratio < 1.0:
        test_X = reshape(test_X, feature_window)
        y_pred_encoder = predict(encoder, test_X)
        y_pred_forecaster = predict(forecaster, y_pred_encoder)
        evaluate(y_pred_forecaster, test_y)
    model = {'scaler': scaler, 'encoder': encoder, 'forecaster': forecaster}
    pickle.dump(model, open(os.path.join(result_path, 'model.pkl'), 'wb'))
    
    return model


def workflow_predict(df, model):
    from numpy.random import seed
    seed(1)
    import tensorflow
    tensorflow.random.set_seed(2)
    
    _, test_X = feature_engineering(df, stage='PREDICT', model=model)
    test_X = reshape(test_X.values, feature_window=60)
    print("after reshape: ", test_X)
    y_pred_encoder = predict(model['encoder'], test_X)
    print("After encoder: ", y_pred_encoder)
    y_pred_forecaster = predict(model['forecaster'], y_pred_encoder)
    print("After forecaster: ", y_pred_forecaster)
    return y_pred_forecaster

In [8]:
# 7. Parameters
train_ratio=0.99
feature_window=60
target_window=1
lead_time_window=0
result_path='/Users/zhang_family_mac/Yongqiang/stock_prediction/result'

In [10]:
# 8. Main function
def main():
    workflow_train(train_ratio=train_ratio,
                   feature_window=feature_window,
                   target_window=target_window,
                   lead_time_window=lead_time_window,
                   result_path=result_path)

In [11]:
if __name__ == '__main__':
    main()

scaled:  (8881, 15)
    var1(t-60)  var2(t-60)  var3(t-60)  var4(t-60)  var5(t-60)  var6(t-60)  \
60    0.000272    0.000826    0.008802    0.000016    0.000476    0.001206   
61    0.000827    0.000891    0.013548    0.000787    0.000812    0.001903   
62    0.000893    0.000966    0.010271    0.000634    0.000675    0.001402   
63    0.000968    0.000654    0.007902    0.000596    0.000613    0.001179   
64    0.000655    0.001190    0.008628    0.000381    0.000836    0.001006   
65    0.001192    0.001271    0.011267    0.001105    0.001203    0.001786   
66    0.001273    0.001215    0.011014    0.001184    0.001194    0.001460   
67    0.001217    0.001648    0.011949    0.001218    0.001337    0.001464   
68    0.001651    0.001763    0.008872    0.001605    0.001471    0.001645   
69    0.001767    0.001240    0.007780    0.001118    0.001409    0.001570   

    var7(t-60)  var8(t-60)  var9(t-60)  var10(t-60)  ...  var7(t-1)  \
60    0.001786    0.005040    0.001113     0.00156

Epoch 40/50
 - 8s - loss: 6.8865e-04 - mse: 6.8865e-04
Epoch 41/50
 - 8s - loss: 7.0580e-04 - mse: 7.0580e-04
Epoch 42/50
 - 8s - loss: 6.7818e-04 - mse: 6.7818e-04
Epoch 43/50
 - 8s - loss: 6.7854e-04 - mse: 6.7854e-04
Epoch 44/50
 - 9s - loss: 6.9984e-04 - mse: 6.9985e-04
Epoch 45/50
 - 8s - loss: 6.7589e-04 - mse: 6.7589e-04
Epoch 46/50
 - 8s - loss: 6.6339e-04 - mse: 6.6339e-04
Epoch 47/50
 - 8s - loss: 6.5206e-04 - mse: 6.5206e-04
Epoch 48/50
 - 8s - loss: 6.6753e-04 - mse: 6.6753e-04
Epoch 49/50
 - 8s - loss: 6.8138e-04 - mse: 6.8138e-04
Epoch 50/50
 - 8s - loss: 6.5484e-04 - mse: 6.5484e-04
Evaluation result - rmse:  [nan]  mape:  0.07858445


# Only SP500
df = pd.read_csv('/Users/zhang_family_mac/Yongqiang/stock_prediction/data/sp500_max.csv')
df.columns = map(str.lower, df.columns)
df = df[['open', 'close', 'volume', 'low', 'high']]

high_max = df['Close'].max()
high_min = df['Close'].min() 
y_pred = y_pred * (high_max - high_min) + high_min
df_final = pd.DataFrame({'y_pred': y_pred.ravel(), 'y_true': df.loc[60:, 'Close'].values})
df_final['diff'] = df_final['y_pred'] - df_final['y_true']
import math
rmse = math.sqrt((df_final['diff'] * df_final['diff']).mean())
print('RMSE: ', rmse)
error_ratio = rmse / df_final['y_true'].mean()
print('error ratio: ', error_ratio)